<a href="https://colab.research.google.com/github/adisharma29/2015/blob/master/use.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
#@title Load the Universal Sentence Encoder's TF Hub module
!pip install bertopic
!pip install keybert[use]
!pip install keyphrase-vectorizers
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached typing_extensions-4.2.0-py3-none-any.whl (24 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.0.17 requires typing-extensions<4.2.0,>=3.7.4.1; python_version < "3.8", but you have typing-extensions 4.2.0 which is incompatible.
spacy 3.3.0 requires typing-extensions<4.0.0.0,>=3.7.4; python_version < "3.8", but you have typing-extensions 4.2.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-

module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [33]:
df_1000 = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-0-1000.xlsx')
df_2000 = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-1000-2000.xlsx')
df_3000 = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-2000-3000.xlsx')
df_5000 = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-3000-5000.xlsx')
df_7000 = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-4865-7000.xlsx')
df_9000 = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-7000-9000.xlsx')
df_last = pd.read_excel('drive/MyDrive/ta/processed/final/review_details-9000-13431.xlsx')
# vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)

In [34]:
df = pd.concat([df_1000, df_2000, df_3000, df_5000, df_7000, df_9000, df_last ], axis=0)

In [11]:
df.head()

,parent_url,review_url,meta,review_title,travel_date,traveller_type,review_detail,written,review_detail.1,review_date
0,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-or10-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,"mulleeozSydney, Australia8 contributions0",Never disappoints!,2022-03-01,NaN,"First thing in the morning or at sunset are the most peaceful times to enjoy this magnificent icon of India, a monument to love.Read more",2022-05-01,NaN,NaN
1,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-or10-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,"emanuellandellStockholm, Sweden54 contributions0",Cinematic scenery from morning to evening,2022-05-01,Solo,"It is a very beautiful scenery with amazing engineering, architecture and hand crafting. Come early and watch it spark in the sunrise.Read more",2022-04-30,NaN,NaN
2,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-or10-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,"Masher16Mumbai, India2,470 contributions0",Simply mesmerising!,2022-04-01,NaN,What do I say about the Taj Mahal that has already not been said?! Took me so many years to finally visit it even though I live in India. Was completely blown over by the magnificence and boundless beauty of this spectacular monument and intrigued by the glorious as well as cruel history it holds. It's best to book a ticket online to avoid long queues to purchase one. There will be queues even...,2022-04-27,NaN,NaN
3,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-or10-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,"dr floPort Saint Lucie, FL2 contributions0","Amazing adventure, great country, food was great and the people super attentive",2022-04-01,Family,"Great country, excellent food, people super attentive. Monument poorly kept but still standing. Tour where amazing and full of education.Read more",2022-04-26,NaN,NaN
4,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,https://tripadvisor.in/Attraction_Review-g297683-d317329-Reviews-Taj_Mahal-Agra_Agra_District_Uttar_Pradesh.html,Curiositytraveller12 contributions0,Real wonder,2022-05-01,Couples,It is must see monument. You can buy ticket online. It is a big complex with beautiful marble building and garden. It is truly one of the wonder of the world.Read more,2022-05-30,NaN,NaN


In [35]:
df2 = df.groupby(['parent_url'])['review_detail'].apply(lambda x: '\n\n'.join([str(y) for y in x])).reset_index()

In [ ]:
df2

,parent_url,review_detail
0,https://tripadvisor.in/Attraction_Review-g1005...,This is Swayambhu Venkateshwara swamy Temple l...
1,https://tripadvisor.in/Attraction_Review-g1005...,Khanapuram Mini tank bund is located on bypass...
2,https://tripadvisor.in/Attraction_Review-g1005...,Lakaram Lake is one of the lake which is locat...
3,https://tripadvisor.in/Attraction_Review-g1005...,I came to India for my marriage (American marr...
4,https://tripadvisor.in/Attraction_Review-g1005...,"We came in January, so the dam wasn't open (me..."
...,...,...
13268,https://tripadvisor.in/Attraction_Review-g9734...,Kuppam river is the wedding of a river and bac...
13269,https://tripadvisor.in/Attraction_Review-g9734...,In Kannur district we have a lot to say to tho...
13270,https://tripadvisor.in/Attraction_Review-g9740...,Definitely must visit place on trip to madurai...
13271,https://tripadvisor.in/Attraction_Review-g9822...,It had a very good view point at this place. S...


In [ ]:
# list_review =  df2['review_detail'].tolist()

In [ ]:
# embeddings = embed(list_review)

In [ ]:
np_arr = []
for idx, review_row in df2.iterrows():  
  np_arr.append(embed([review_row['review_detail']])[0])

In [ ]:
embeddings = np.stack(np_arr)

In [ ]:
# embeddings

In [ ]:
# scores = []
# for idx, review_row in df2.iterrows():
#   # review = review_row[1]  
#   review_emb = embed([review_row['review_detail']])
#   score = np.inner(sentence_embedding, review_emb)[0][0]
#   scores.append(score)


In [ ]:
# idxes = sorted(range(len(scores)), key = lambda sub: scores[sub])[-5:]
# print([scores[i] for i in idxes])

In [ ]:
# print(df2['parent_url'][idxes])

In [ ]:
sentence_embedding = embed(['a luxury place for couples with nice weather, a romantic setting and wine. A moonlit walk would be an added advantage'])
similarity_array = np.inner(sentence_embedding, embeddings)[0]
top = similarity_array.argsort()[::-1][:10]
print(similarity_array[top])
df2.iloc[top,:]

[0.3944431  0.37529683 0.37312174 0.3382097  0.3292192  0.32749063
 0.32238743 0.32117227 0.31516552 0.31486386]


,parent_url,review_detail
3257,https://tripadvisor.in/Attraction_Review-g2282...,Very attractive and natural tourist place for ...
7855,https://tripadvisor.in/Attraction_Review-g2976...,A must for those who love a little trekking. W...
3453,https://tripadvisor.in/Attraction_Review-g2287...,Dudhawa Dam is built across Mahanadi river is ...
13105,https://tripadvisor.in/Attraction_Review-g8584...,A nice place for an evening stroll with your p...
496,https://tripadvisor.in/Attraction_Review-g1138...,rakasganda fall is a beautiful waterfall on ri...
12003,https://tripadvisor.in/Attraction_Review-g6609...,If you lover of nature then this is good place...
13075,https://tripadvisor.in/Attraction_Review-g8547...,it is good place for tour i and my friend goes...
10963,https://tripadvisor.in/Attraction_Review-g5037...,This is also called Champak Lake. It is a good...
1722,https://tripadvisor.in/Attraction_Review-g1239...,I spent over three hours at the Palm spa- deep...
10882,https://tripadvisor.in/Attraction_Review-g5036...,Once upon a time the Deepor bill was home to l...


In [ ]:
# index_max = np.argmax(similarity_array)
# top_4 = np.argpartition(similarity_array, -10)[-10:]
# top_4 = similarity_array.argsort()[::-1][:4]
# sorted = np.sort(similarity_array[top_4])
# sorted

In [ ]:
df2['embeddings'] = [str(i.tolist()) for i in embeddings]

In [ ]:
df2.head()

,parent_url,review_detail,embeddings
0,https://tripadvisor.in/Attraction_Review-g1005753-d21513975-Reviews-Sri_Venkateswara_Swamy_temple-Khammam_Khammam_District_Telangana.html,"This is Swayambhu Venkateshwara swamy Temple located in Jeela Cheruvu village of Khammam District, which is on Khammam to Suryapeta Highway at a distance of 28 km from Khammam Town.This temple was constructed in 1964 and within 2 years it was adopted by the Endowment department in 1966.Near by Attractions:1. Nelakondapally Venkateshwara Alayam, Budha Stupam, Bakta Ramadasu kshetram2. Kusumanch...","[-0.04686907306313515, 0.0627259835600853, -0.0024402092676609755, -0.06856626272201538, 0.0018221988575533032, 0.017945772036910057, 0.04228108376264572, -0.04722059518098831, 0.03119659796357155, -0.011553816497325897, 0.031830012798309326, 0.03965260833501816, -0.04301382601261139, -0.010827656835317612, 0.055076416581869125, 0.030055560171604156, -0.05171956121921539, -0.024948908016085625..."
1,https://tripadvisor.in/Attraction_Review-g1005753-d23481775-Reviews-Khanapuram_Mini_Tank_Bund-Khammam_Khammam_District_Telangana.html,"Khanapuram Mini tank bund is located on bypass road of Khammam. This is actually normal lake which is developed with 1.2 Cr, in recent days. This is inaugurated by KTR on 7th December 2020.This is best place to spend with kids in the evening. and also there is walking lane arranged inside for morning and evening walkersRead more","[-0.0679028332233429, -0.030843446031212807, 0.05211750417947769, 0.00047578196972608566, -0.0780850425362587, 0.013843120075762272, -0.04608139023184776, -0.01835673861205578, 0.06451243907213211, -0.054844606667757034, -0.0067199040204286575, -0.04810450226068497, -0.0369139090180397, 0.016743117943406105, 0.03559676185250282, 0.06809930503368378, -0.06016929820179939, 0.022091485559940338, ..."
2,https://tripadvisor.in/Attraction_Review-g1005753-d23481776-Reviews-Lakaram_Lake-Khammam_Khammam_District_Telangana.html,Lakaram Lake is one of the lake which is located inside city. it is located 4 km away from Bus stand. 15 years back it was just a lake surrounded by trees. After Telangana formation it is developed under Mission Kakatiya with many tourist attractions surrounding this lake. Now it is a lake with Park and Boating facility and also developed Tank bund surrounding this lake.This is busiest weekend...,"[-0.009210021235048771, 0.01644778996706009, 0.017972463741898537, -0.012027481570839882, 0.02872764691710472, 0.0661000907421112, 0.05083253234624863, -0.046123579144477844, 0.07119345664978027, -0.06435220688581467, -0.006785046309232712, 0.03232250735163689, 0.009185601957142353, -0.06543515622615814, 0.06140080466866493, -0.0013695011148229241, -0.017849313095211983, -0.06260905414819717, ..."
3,https://tripadvisor.in/Attraction_Review-g1005753-d3705296-Reviews-Sri_Sitaramachandra_Swamy_Temple-Khammam_Khammam_District_Telangana.html,"I came to India for my marriage (American marrying Indian), so we visited a lot of temples as it was my first time in the country. This temple was by far my favorite. It was clean, the lines weren't too long, and you had way more time to look at multiple Gods than at Tirumala. Read more\n\nThe trip from chennai involved a rail visit to Rajahmundry and then an hour by road to reach the boat poi...","[-0.05371842533349991, 0.04880871996283531, 0.05237329378724098, -0.05472073331475258, 0.054929543286561966, 0.03925148770213127, 0.04312186315655708, -0.05562471225857735, 0.05691562592983246, -0.05760585889220238, 0.03498023748397827, 0.04719546064734459, -0.05690627917647362, -0.024073448032140732, 0.02004934847354889, -0.05732019618153572, -0.049951788038015366, 0.05359821021556854, -0.051..."
4,https://tripadvisor.in/Attraction_Review-g1005753-d5788009-Reviews-Kinnerasani_Wildlife_Sanctuary-Khammam_Khammam_District_Telangana.html,"We came in January, so the dam wasn't open (meaning the other side of the dam was pretty dry). With that being said, the view was absolu

In [ ]:
df2.to_csv('drive/MyDrive/ta/processed/final/reviews_grouped_emb.tsv', sep='\t')

In [ ]:
print(top_4)
df2.iloc[top_4,:]

[301 906 223  57]


,parent_url,review_detail
301,https://tripadvisor.in/Attraction_Review-g297628-d2587750-Reviews-Ulsoor_Lake-Bengaluru_Bangalore_District_Karnataka.html,"nice ambience and very well maintained interior, one more suggestion is there some out Side Area need to cleaned. Cool environment with good view.Read more\n\nIt's a lovely service n kind hearted ...The food n everything was sooo good..Srija she was suchhh a sweetheart üíóshes so good !!!love the way she was talking wit us.. Alhamdulillah...Thank youuu so much srijaaa...it was lovely talki..."
906,https://tripadvisor.in/Attraction_Review-g659792-d1314871-Reviews-Bharati_Government_Park-Pondicherry_Union_Territory_of_Pondicherry.html,A good clean park. good for walking and kids to play around. lot of trees. though we went in summer we felt goodRead more\n\nbest place to visit with children. lot of play options and activities are lined up for the kids here.Read more\n\nI walk here every time I visit Pondicherry but only this trip did I hear the story of how this park was formed. Very French!Read more\n\nA beautiful garden i...
223,https://tripadvisor.in/Attraction_Review-g297608-d2501233-Reviews-Sabarmati_Riverfront-Ahmedabad_Ahmedabad_District_Gujarat.html,"earlier there was the slum at this place !! But Mr. Modi Chief Minister of State of that time did good and shifted all people to other houses and beautiful riverfront developed !Read more\n\nStaying at the Hyatt Regency, The Riverfront in the evenings was a wonderfully refreshing sight.If you are staying at one of the hotels astriding the Sabarmati, would reccommend at least one visit during t..."
57,https://tripadvisor.in/Attraction_Review-g1162345-d1601651-Reviews-Yercaud-Salem_Salem_District_Tamil_Nadu.html,"This is a must visit hill station in Tamil Nadu. Can spend 2 days easily, take cab for local tours. Lot of view points snd gardens to visit and ofcourse boating. This area produces great pepper and tea, coffee and massage oils and you can buy them everywhere.Read more\n\nYercaud is a nice Tourist place, except on 2 factors... Roads are very bad and Cleanliness is poor as most of the places are..."


In [13]:
pd.set_option('max_colwidth', 400)

In [36]:
from keybert import KeyBERT
from keyphrase_vectorizers import KeyphraseCountVectorizer
import itertools 
# vectorizer = KeyphraseCountVectorizer( stop_words='english')
vectorizer = KeyphraseCountVectorizer( stop_words='english',pos_pattern='<J.*>*<N.*>' )
kw_model_use = KeyBERT(model=model)
kw_model_default = KeyBERT()
kw_model_ad = KeyBERT(model='paraphrase-mpnet-base-v2')

In [37]:
!python -m spacy download en_core_web_md
import spacy
import itertools
from itertools import combinations


# Set globals
nlp = spacy.load("en_core_web_md")

def pre_process(titles):
    """
    Pre-processes titles by removing stopwords and lemmatizing text.
    :param titles: list of strings, contains target titles,.
    :return: preprocessed_title_docs, list containing pre-processed titles.
    """

    # Preprocess all the titles
    title_docs = [nlp(x) for x in titles]
    preprocessed_title_docs = []
    lemmatized_tokens = []
    for title_doc in title_docs:
        for token in title_doc:
            if not token.is_stop:
                lemmatized_tokens.append(token.lemma_)
        preprocessed_title_docs.append(" ".join(lemmatized_tokens))
        del lemmatized_tokens[
            :
            ]  # empty the lemmatized tokens list as the code moves onto a new title

    return preprocessed_title_docs

def similarity_filter(titles):
    """
    Recursively check if titles pass a similarity filter.
    :param titles: list of strings, contains titles.
    If the function finds titles that fail the similarity test, the above param will be the function output.
    :return: this method upon itself unless there are no similar titles; in that case the feed that was passed
    in is returned.
    """

    # Preprocess titles
    preprocessed_title_docs = pre_process(titles)

    # Remove similar titles
    all_summary_pairs = list(combinations(preprocessed_title_docs, 2))
    similar_titles = []
    for pair in all_summary_pairs:
        title1 = nlp(pair[0])
        title2 = nlp(pair[1])
        similarity = title1.similarity(title2)
        if similarity > 0.8:
            similar_titles.append(pair)

    titles_to_remove = []
    for a_title in similar_titles:
        # Get the index of the first title in the pair
        index_for_removal = preprocessed_title_docs.index(a_title[0])
        titles_to_remove.append(index_for_removal)

    # Get indices of similar titles and remove them
    similar_title_counts = set(titles_to_remove)
    similar_titles = [
        x[1] for x in enumerate(titles) if x[0] in similar_title_counts
    ]

    # Exit the recursion if there are no longer any similar titles
    if len(similar_title_counts) == 0:
        return titles

    # Continue the recursion if there are still titles to remove
    else:
        # Remove similar titles from the next input
        for title in similar_titles:
            idx = titles.index(title)
            titles.pop(idx)
            
        return similarity_filter(titles)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.5 MB 201 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:



def get_keywords_for_reviews(reviews):
  num_reviews = len(reviews) 
  final_tags_dict = {} 
  sorted_tags_dict = {}
  for review_text in reviews:    
    arr_use = kw_model_use.extract_keywords(review_text, vectorizer=vectorizer, use_mmr=True, top_n=5)    
    if arr_use is not None:
      for phrase, score in arr_use:

        if final_tags_dict.get(phrase) is not None:
          final_tags_dict[phrase] += 1
        else:
          if num_reviews < 10:
            if score > 0.1:
              final_tags_dict[phrase] = score
          else:
            if score > 0.2:
              final_tags_dict[phrase] = score
      sorted_tags_dict = {k: v for k, v in sorted(final_tags_dict.items(), key=lambda item: item[1],reverse = True)}
  top_20 = dict(itertools.islice(sorted_tags_dict.items(), 20))
  non_similar = similarity_filter(list(top_20.keys()))  
  return non_similar

for idx, parent_url in df2.iloc[612:,:]['parent_url'].iteritems():
  keywords_string = ''
  keywords_list = []
  dfx = df[df['parent_url'] == parent_url]
  review_list = dfx['review_detail'].tolist()
  keywords_list = get_keywords_for_reviews(review_list)
  keywords_string = ','.join(keywords_list) 
  with open('drive/MyDrive/ta/tags/tag_file.tsv', 'a') as the_file:
    print(idx, keywords_string)
    the_file.write(str(idx) + '\t' + parent_url + '\t' + keywords_string + '\n')
# dfx = df[df['parent_url'] == 'https://tripadvisor.in/Attraction_Review-g1162480-d14797656-Reviews-Karukathamman_Temple-Mahabalipuram_Kanchipuram_District_Tamil_Nadu.html'] 
# get_keywords_for_reviews(dfx['review_detail'])

612 ji,serene ambience,best hanuman,bhagwan,short duration,km,divine,corbett,positive energy,safari,older incarnations,ideal
613 corbett,resort,picnic,nature,riverside,temple,jim,mahindra,clear water,fresh water,cold water,nice river,birds,clean water,national park,swimming,nice place,beautiful view


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


614 corbett,safari,sita,jungle,monkeys,nothing,buffer,jim,nice place,visit,ancient temple,deer,mata,dense forest,jeep
615 safari,elephants,national park,wild animals,park,gypsy,deers,jungle,rhino,guides,peacock,dense forest,reserve,tower


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


616 foreign tourist,hindu,calm places,sculpture,architect,balaji,nice temple,peace,community,design,nothing
617 lovely fruits,potatoes,apples,decent trek,aug,nice place,fruits,shimla,vegetables,day,unique e3xperience
618 activities,village,family,welcome drink,culture,camel,nature,authentic north,traditional food,north karnataka,ride,food,traditional culture,good ambiance,beautiful place
619 temple,spiritual books,revered saint,hungry ones,swami,free food,spiritual place,karnataka,best place,rich culture,preaching,devotion,local shivaite guru,famous place,donations,shri,swamy
620 saturday,kms,lake,dharwad,famous hanuman,dosa,daily devotees,best temple,nice hanuman,spiritual vibrance,religious fervor,itinerary,local shops,ji,great view,great place,picnic
621 activities,dharwad,indoor activities,adventures,populated world,hospitality,outdoor activities,fears,cycling,best theme,ultimate form,cylindrical zorb,friends,karnataka,many time
622 peaceful place,lot
623 shiva,unknown monument,act

...

650 darshan,oldest temple,little difficult crowded place,mata,visit,deity,devine presence,full devotion,pathankot,devotional people,holy place,market,.governor,legend,shops
651 mandir,tuesday,kali,sacred temple,old temple,devotees,ancient templeread,ka,path,governor,main entrance,distributor,cultural activities,dalhousie
652 vishnu,laxmi,huge statue,good temple,mandir,family,grand father,main temple,good situated location,brass,goddess,gate,governor,good .my,continuous long life,amazing architecture,store,green surroundings
653 shani,temple,famous temples,famous shani,cousin,famous worship,devotion,whole pathankot,himalayas,beautiful natural settings,followers,rush,awesome scenic beauty,aura,darshan,visit,saturday,nyc
654 dam,sagar,bridge,beautiful boat,special permision,maharaja,visit,forest,excessive rains,picnic,wild rabbits,lovely green scenes,wire,magnificent landscape,humongous lake,joints
655 singh,fort,ravi,durga,famous temple,great mughal emperor,many beautiful mosques,maharaj

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


657 caves,mahabharata,hindus,parvati,shiva,calm breeze,natural caves,stairs,ancient temple,historical place,pandavas,brothers,pandit,inner peace,refuge


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:50: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.


658 himachal,snake,km,bari nagni,beautiful temple,nagni,goddess,famous attractions,healing,local folklore,power,spiritual aspect,mata,thakur,particular area,full devotion
659 civil engineering,engineers,direct interest,canal,researches,floods,dams,institution,center,commissioner,models
660 elephants,safari,single elephants,poor visibility,deers,first few jeeps,trees,excellent wild animal,monsoons,early bird,forest,tiger,reserve,pathetic jeep,early morning,entry


In [ ]:
review_text = df2['review_detail'][1342]
# arr = kw_model.extract_keywords(review_text, keyphrase_ngram_range=(1,2), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)
arr_use = kw_model_use.extract_keywords(review_text, keyphrase_ngram_range=(1,3), use_mmr=True, diversity=0.5, top_n=15)
# arr_default = kw_model_default.extract_keywords(review_text, keyphrase_ngram_range=(1,3), use_mmr=True, diversity=0.5, top_n=15)
[print(a) for a in arr_use]
print("-"*50)
# print(review_text)

In [ ]:
# df2['keywords'] = df2['review_detail'].apply(lambda z: kw_model.extract_keywords(z, keyphrase_ngram_range=(1, 2), stop_words=None)) 
arr_default = kw_model_flair.extract_keywords(review_text, keyphrase_ngram_range=(1,3), use_mmr=True, diversity=0.5, top_n=15)
arr_default

In [ ]:
dfx = df[df['parent_url'] == 'https://tripadvisor.in/Attraction_Review-g1162480-d14797656-Reviews-Karukathamman_Temple-Mahabalipuram_Kanchipuram_District_Tamil_Nadu.html'] 

In [ ]:
from keyphrase_vectorizers import KeyphraseCountVectorizer
# vectorizer = KeyphraseCountVectorizer( stop_words='english')
vectorizer = KeyphraseCountVectorizer( stop_words='english',pos_pattern='<J.*>*<N.*>' )



In [ ]:
# vectorizer = CountVectorizer(ngram_range=(1, 3), stop_words="english")
for _, review_text in dfx['review_detail'].iteritems():
  arr_use = kw_model_use.extract_keywords(review_text, vectorizer=vectorizer, use_mmr=True, top_n=5)
  print(sorted(arr_use, key=lambda x: x[1], reverse=True))
  # arr_use = kw_model_use.extract_keywords(review_text, keyphrase_ngram_range=(1,3), top_n=5)
  # arr_use = kw_model_use.extract_keywords(review_text, vectorizer=vecrotizer, keyphrase_ngram_range=(1,2), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)

2022-06-03 18:24:27,089 - KeyphraseVectorizer - INFO - It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
INFO:KeyphraseVectorizer:It looks like you do not have downloaded a list of stopwords yet. It is attempted to download the stopwords now.
/usr/local/lib/python3.7/dist-packages/spacy/util.py:845: UserWarning: [W094] Model 'en_core_web_sm' (2.2.5) specifies an under-constrained spaCy version requirement: >=2.2.2. This can lead to compatibility problems with older versions, or as new spaCy versions are released, because the model may say it's compatible when it's not. Consider changing the "spacy_version" in your meta.json to a version range, with a lower and upper pin. For example: >=3.3.0,<3.4.0
  warnings.warn(warn_msg)
2022-06-03 18:24:27,223 - KeyphraseVectorizer - INFO - It looks like the selected spaCy pipeline is not downloaded yet. It is attempted to download the spaCy pipeline now.
2022-06-03 18:24:27,223 - Keyph

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
[('goddess', 0.2926), ('barren womb', 0.2535), ('amman', 0.2512), ('safeguards', 0.1458), ('similar instances', 0.068)]
[('temple', 0.4964), ('hindu', 0.3345), ('calm place', 0.1196), ('place', 0.1056)]
[('temples', 0.254), ('local culture', 0.2302), ('mahabalipuram', 0.149), ('best time', 0.14), ('friday', 0.0941)]
[('sculptures', 0.2798), ('temple', 0.2704), ('amman', 0.2439), ('blessings', 0.1726), ('lovable environment', 0.1182)]
[('temple', 0.2378), ('religious traveler', 0.2284), ('sunset', 0.2252), ('mahabalipuram', 0.177), ('true believer', 0.1471)]


In [ ]:
review_text = df2['review_detail'][1224]
df2['parent_url'][1224]

'https://tripadvisor.in/Attraction_Review-g1162480-d14797656-Reviews-Karukathamman_Temple-Mahabalipuram_Kanchipuram_District_Tamil_Nadu.html'

In [ ]:
arr_use = kw_model_use.extract_keywords(review_text, vectorizer=vectorizer, use_mmr=True, top_n=40)
# [print(a) for a in arr_use]
sorted(arr_use, key=lambda x: x[1], reverse=True)

In [ ]:
from bertopic import BERTopic

In [ ]:
topic_model = BERTopic(embedding_model=model)
topics, probs = topic_model.fit_transform(df['review_detail'].dropna().apply(lambda x : str(x)).tolist())

In [ ]:
topic_model.get_topic_info()

NameError: ignored

In [ ]:
topic_model.visualize_topics()

NameError: ignored

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(2, 3))

In [ ]:
topic_model.update_topics(df2['review_detail'].tolist(), topics, vectorizer_model=vectorizer_model)

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name
0,-1,4012,-1_temple_place_read_visit
1,0,736,0_shop_market_shopping_quality
2,1,477,1_garden_park_place_read
3,2,421,2_beach_sea_clean_beaches
4,3,409,3_falls_waterfall_water_fall
...,...,...,...
123,122,11,122_netaji_house_tagore_kalam
124,123,10,123_narasimha_badri_lord_temple
125,124,10,124_ghats_monsoon_malshej_ghat
126,125,10,125_house_wing_braganza_portuguese


In [ ]:
topic_model.get_topic(37)[-20:]

[('ganga aarti', 0.011641912981971559),
 ('boat ride', 0.006652731152118561),
 ('har ki', 0.0061231052463358426),
 ('assi ghat', 0.005962147561697594),
 ('ki pauri', 0.005222331212511972),
 ('har ki pauri', 0.005011111337512798),
 ('river ganga', 0.004682525273351725),
 ('triveni sangam', 0.004322922530043107),
 ('evening aarti', 0.004200095242097189),
 ('ganga river', 0.003943364860755272)]